In [ ]:
from os import path
import numpy as np
from itertools import product
import matplotlib.pyplot as plt

from brainpipe.system import study
from brainpipe.feature import erpac
from brainpipe.feat.utils._feat import _manageWindow
from brainpipe.visual import *
from tensorpac import Pac

plt.style.use('seaborn-poster')

### Compute and Plot PAC values 
    Low frequency phases and large broad band gamma amplitude

In [ ]:
st = study('Olfacto')
Test = False
if Test == True:
    subjects = ['FERJ']
    conds = {'subj':('LowHigh',['low','high'])}
else:
    subjects = ['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']
    conds = {'subj':('LowHigh',['low','high'])}

### Compare different PAC methods and normalization

In [ ]:
for su, cond in product(subjects[:], sorted(conds)):
    pathdata = path.join(st.path,'database/Encoding_EpiPerf_'+conds[cond][0]+'/')
    path2save = path.join(st.path,'feature/2_PAC_Encoding_EpiPerf_'+conds[cond][0]+'/ERPAC/')
    
    for c in conds[cond][1]:
        fi = su+'_odor_'+c+'_bipo_sel_physFT.npz'
        loadname = pathdata+fi
        if path.isfile(loadname):
            mat = np.load(loadname)
            print(mat.files)
            data = mat['x'][:,:2304,:].swapaxes(1,2)
            labels = mat['Mai_RL']
            time = np.arange(-1000, 2500, 100)
            n_elec, n_trials, npts = data.shape
            sf, famp = 512., (60,120,5,1)
            width, step = 358, 51
            fpha = [[0.1,1.5],[2,3],[3,7],[8,13]]
            fnames = ['VLFC', 'delta', 'theta', 'alpha']
            print ('--> compute pac on : ', fi, 'data', data.shape, 'npts', npts, 'ntrials',n_trials, 'nelec', n_elec)              
            
            for elec in range(n_elec):
                label = labels[elec]
                data_elec = data[elec,:,:][np.newaxis].swapaxes(1,2)
                # Define an ERPAC object :
                print(data_elec.shape)
                erpacObj = erpac(sf, npts, pha_f=fpha, pha_meth='hilbert', pha_cycle=3, 
                                 amp_f=famp, amp_meth='hilbert',amp_cycle=6,step=step, 
                                 width=width)
                
                # Compute the ERPAC - traxis specify which is the trial axis
                pac, pval = erpacObj.get(data_elec, data_elec, n_perm=200, n_jobs=-1)
                pac, pval = np.squeeze(pac), np.squeeze(pval)
                print(pac.shape, pval.shape)
                
                for pha in range(pac.shape[0]):
                    # Plot without p-values :
                    fig = plt.figure()
                    title = 'ERPAC for freqs '+fnames[pha]+' ['+str(fpha[pha])+'] / High-Gamma '+su+' '+c+' '+str(label)+' ('+str(elec)+')'
                    Pac.pacplot(erpacObj,pac=pac[pha,:,:], xvec=time, yvec=[60,120], xlabel='Time (ms)', 
                            cmap='inferno',ylabel='Amplitude frequency', title=title, cblabel='ERPAC',
                            vmin=0., rmaxis=True, interp=(.1,.1))
                    addLines(plt.gca(),vLines=[0], vColor=['white'], vShape=['-'], vWidth=[2.])
                    filename = su+'_ERPAC_'+fnames[pha]+'_gamma_'+label+'_('+str(elec)+')_'+c+'.png'
                    plt.tight_layout()
                    plt.savefig(path2save+filename)
                    plt.clf()
                    plt.close()